# 실습 2
mortdefault2009.csv는 2009년 모기지론과 관련한 데이터입니다.
Feature의 수는 5개입니다. default 변수의 경우, default 변수는 정상0, default 발생 1의 값을 갖고 있습니다.

주어진 데이터를 활용하여, creditScore에 대한 Regression을 적용해보세요.
1.	데이터 읽기
2.	Missing 확인 및 처리, @@@ 또는 qqq 등 이상한 값 제거
3.	Training과 Testing으로 8:2로 파티셔닝 / Data Scaling은 필요 시 수행
4.  default가 1인 값들만 남기기
5.	"creditScore"는 Y로,"houseAge","yearsEmploy"를 X변수로 하는 regression을 수행, Testing 셋에 대한 rmse를 계산


In [22]:
import pandas as pd 
import numpy as np

md = pd.read_csv("data/mortDefault.zip")
md.info()
md.value_counts() / md.shape[0]


/var/folders/bx/zv234tzj43l4hqdbys0hclvw0000gn/T/ipykernel_38797/1992156137.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv("data/mortDefault.zip")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   creditScore  1000000 non-null  object 
 1   houseAge     999999 non-null   float64
 2   yearsEmploy  999999 non-null   float64
 3   ccDebt       1000000 non-null  int64  
 4   year         1000000 non-null  int64  
 5   default      1000000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 45.8+ MB


creditScore  houseAge  yearsEmploy  ccDebt  year  default
678          22.0      4.0          0       2009  0          0.000005
705          20.0      4.0          0       2009  0          0.000004
695          18.0      5.0          0       2009  0          0.000004
689          20.0      5.0          0       2009  0          0.000004
670          21.0      4.0          0       2009  0          0.000003
                                                               ...   
685          23.0      10.0         2922    2009  0          0.000001
             24.0      0.0          2429    2009  0          0.000001
                                    4793    2009  0          0.000001
                                    5116    2009  0          0.000001
qqq          11.0      5.0          5618    2009  0          0.000001
Name: count, Length: 997969, dtype: float64

In [23]:

md.drop("year", axis=1, inplace=True)
md1 = md[md.default == 1]
md1.drop('default',axis=1, inplace=True)
md1.shape

md1.replace({'@@@':np.nan, 'qqq':np.nan}, inplace=True)
md1.dropna(inplace=True)


md1.creditScore = md1.creditScore.astype('int')


/var/folders/bx/zv234tzj43l4hqdbys0hclvw0000gn/T/ipykernel_38797/4057760190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  md1.drop('default',axis=1, inplace=True)
/var/folders/bx/zv234tzj43l4hqdbys0hclvw0000gn/T/ipykernel_38797/4057760190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  md1.replace({'@@@':np.nan, 'qqq':np.nan}, inplace=True)
/var/folders/bx/zv234tzj43l4hqdbys0hclvw0000gn/T/ipykernel_38797/4057760190.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [24]:
from sklearn.preprocessing import StandardScaler
sc1 = StandardScaler().fit(md1[["creditScore", "houseAge", "yearsEmploy"]])
sc1 = sc1.transform(md1[["creditScore", "houseAge", "yearsEmploy"]])


In [ ]:
import seaborn as sns
sns.heatmap( md1.corr() )

In [29]:
from sklearn.preprocessing import StandardScaler
scaler_1 = StandardScaler().fit( md1[ ["creditScore", "houseAge","yearsEmploy"]] )
scaled_1 = scaler_1.transform( md1[ ["creditScore", "houseAge","yearsEmploy"]] )
data_1_scaled = pd.DataFrame( scaled_1, columns = ["creditScore", "houseAge","yearsEmploy"] )


In [30]:

from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split( data_1_scaled.drop( "creditScore", axis=1), data_1_scaled.creditScore, test_size=0.2)


In [35]:

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

model_1 = LinearRegression().fit( X_train_1, y_train_1)
#model_1 = DecisionTreeRegressor().fit( X_train_1, y_train_1)
#model_1 = RandomForestRegressor(n_estimators=100).fit( X_train_1, y_train_1)
#model_1 = AdaBoostRegressor().fit( X_train_1, y_train_1)

model_1.score( X_train_1, y_train_1 )

#예측

pred_1 = model_1.predict( X_test_1)
print( "predicted: ", pred_1[0] * scaler_1.scale_[0] + scaler_1.mean_[0],  ", actual:", y_test_1* scaler_1.scale_[0] + scaler_1.mean_[0] )

from sklearn.metrics import mean_squared_error
print( mean_squared_error( pred_1, y_test_1 ) * scaler_1.scale_[0] + scaler_1.mean_[0] )

predicted:  689.0929111809487 , actual: 14257    781.0
1289     748.0
23397    764.0
4597     719.0
20566    624.0
         ...  
10038    693.0
17653    719.0
5029     677.0
2727     654.0
24598    704.0
Name: creditScore, Length: 4949, dtype: float64
736.8101856730129


In [36]:
# 중요 변수를 추출하기 위해서 statsmodels로 변수들의 P-value를 확인한다.
import statsmodels.api as sm

X = sm.add_constant(X_train_1)
y = y_train_1

model = sm.OLS(y,X) # 순서 조심, y먼저
results = model.fit()
print(results.summary())
results.pvalues

                            OLS Regression Results                            
Dep. Variable:            creditScore   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.2682
Date:                Fri, 15 Sep 2023   Prob (F-statistic):              0.765
Time:                        12:00:20   Log-Likelihood:                -28168.
No. Observations:               19792   AIC:                         5.634e+04
Df Residuals:                   19789   BIC:                         5.637e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0055      0.007     -0.770      

const          0.441390
houseAge       0.921176
yearsEmploy    0.469013
dtype: float64